In [69]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy.sparse import hstack

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [71]:
# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Read Data for the Fraudulent Email Kaggle Challenge
# Reduce the training set to speed up development
data = pd.read_csv("../data/kg_train.csv", encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(f"Data shape: {data.shape}")
data.fillna("", inplace=True)

[nltk_data] Downloading package stopwords to C:\Users\Nekky
[nltk_data]     Lung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Nekky
[nltk_data]     Lung\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Nekky
[nltk_data]     Lung\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Data shape: (1000, 2)


### Let's divide the training and test set into two partitions

In [72]:
# Split the data into training and validation sets
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])
print(f"Training set size: {data_train.shape}")
print(f"Validation set size: {data_val.shape}")


Training set size: (800, 2)
Validation set size: (200, 2)


## Data Preprocessing

In [73]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [74]:
# HTML CLEANING SECTION
import re
from bs4 import BeautifulSoup
# TEXT PREPROCESSING SECTION
def preprocess_text(text):
    """Complete text preprocessing pipeline"""
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Remove all special characters (keep only letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b' (if any)
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower().strip()
    
    return text

# Apply text preprocessing
data_train['preprocessed_text'] = data_train['cleaned_text'].apply(preprocess_text)
data_val['preprocessed_text'] = data_val['cleaned_text'].apply(preprocess_text)



KeyError: 'cleaned_text'

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
# show the differences
for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Before: {data_train['cleaned_text'].iloc[i][:100]}...")
    print(f"After:  {data_train['preprocessed_text'].iloc[i][:100]}...")


Example 1:
Before: Dear=2C Good day hope fine=2Cdear am writting this mail with due respect and heartful of tears since...
After:  dearc good day hope finecdear am writting this mail with due respect and heartful of tears since we ...

Example 2:
Before: FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGEFOREIGN REMITTANCE UNIT,AFRICAN DEVELOPMENT BANK{ADB}....
After:  from mr henry kaborethe chief auditor inchargeforeign remittance unitafrican development bankadbouag...

Example 3:
Before: Will do....
After:  will do...

Example 4:
Before: FROM THE DESK OF DR.ADAMU  ISMALERAUDITING AND ACCOUNTING MANAGER,BANK OF AFRICA (B.O.A)OUAGADOUGOU-...
After:  from the desk of dradamu ismalerauditing and accounting managerbank of africa boaouagadougouburkina ...

Example 5:
Before: Dear Friend, My name is LOI C.ESTRADA,The wife of Mr. JOSEPHESTRADA, the former President of Philipp...
After:  dear friend my name is loi cestradathe wife of mr josephestrada the former president of philippines ...

Exam

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
# STOPWORDS REMOVAL SECTION
def remove_stopwords(text):
    """Remove stopwords from text"""
    if not text:
        return ""
    
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply stopwords removal
data_train['no_stopwords'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_val['no_stopwords'] = data_val['preprocessed_text'].apply(remove_stopwords)


print(f"Before stopwords removal: {data_train['preprocessed_text'].iloc[0]}")
print(f"After stopwords removal: {data_train['no_stopwords'].iloc[0]}")

Before stopwords removal: dearc good day hope finecdear am writting this mail with due respect and heartful of tears since we have not known or met ourselves previously am asking for your assistanceci have will be very glad if you can render me assistance to my situation nowe will make my proposal well known if am given the opportunitye would like to use this opportunity to introduce myself to youe am miss johana johnpaul years old girl from liberia cthe only daughter of late godwin johnpaul the deputy minister of national security under the leadership of president charles taylor of liberia who is now in exile after many innocent soul were killede my father was killed by the government of charlestaylorcheaccuse my father of coup attempt and after month my mother cynthia was also killede the main reason why am contacting you now is to seek for your assistance in the area of my future investment and also want to hand over some huge amount of money to youe this money is ten millon five hu

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [ ]:
# LEMMATIZATION SECTION

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    """Apply lemmatization to reduce words to their base form"""
    if not text:
        return ""
    
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Apply lemmatization
data_train['lemmatized_text'] = data_train['no_stopwords'].apply(lemmatize_text)
data_val['lemmatized_text'] = data_val['no_stopwords'].apply(lemmatize_text)

# Update preprocessed_text to include all cleaning steps
data_train['preprocessed_text'] = data_train['lemmatized_text']
data_val['preprocessed_text'] = data_val['lemmatized_text']


print(f"Final preprocessed text sample: {data_train['preprocessed_text'].iloc[0]}")

Final preprocessed text sample: dearc good day hope finecdear writting mail due respect heartful tear since known met previously asking assistanceci glad render assistance situation nowe make proposal well known given opportunitye would like use opportunity introduce youe miss johana johnpaul year old girl liberia cthe daughter late godwin johnpaul deputy minister national security leadership president charles taylor liberia exile many innocent soul killede father killed government charlestaylorcheaccuse father coup attempt month mother cynthia also killede main reason contacting seek assistance area future investment also want hand huge amount money youe money ten millon five hundred thousand u dollar deposited year ago father made sole beneficiaryfnext kin moneye asking stand behalf make claim bankcam girl young age cannt handle transactionci want stand foregin partner oversea also help investment money well send telephone number picture okeeee original copy document concering money 

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# BAG OF WORDS - EXPLORATORY DATA ANALYSIS
# the 10 top words in ham and spam messages (EXPLORATORY DATA ANALYSIS)
def get_top_words(data, label, n=10):
    """
    Get top n words for a specific label
    """
    label_data = data[data['label'] == label]
    all_text = ' '.join(label_data['preprocessed_text'].tolist())
    words = all_text.split()
    word_freq = pd.Series(words).value_counts()
    return word_freq.head(n)

print("\nTop 10 words in HAM messages:")
print(get_top_words(data_train, 0))

print("\nTop 10 words in SPAM messages:")
print(get_top_words(data_train, 1))





Top 10 words in HAM messages:
would        91
president    90
u            88
percent      76
call         75
state        72
work         70
mr           70
one          62
time         60
Name: count, dtype: int64

Top 10 words in SPAM messages:
money          708
account        597
bank           571
fund           542
u              441
business       384
transaction    327
country        320
million        306
company        300
Name: count, dtype: int64


## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,cleaned_text,preprocessed_text,no_stopwords,lemmatized_text,money_mark,suspicious_words,text_len
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,Dear=2C Good day hope fine=2Cdear am writting ...,dearc good day hope finecdear writting mail du...,dearc good day hope finecdear writting mail du...,dearc good day hope finecdear writting mail du...,1,1,1024
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,mr henry kaborethe chief auditor inchargeforei...,mr henry kaborethe chief auditor inchargeforei...,mr henry kaborethe chief auditor inchargeforei...,0,1,1954
971,Will do.,0,Will do.,,,,0,0,0
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,desk dradamu ismalerauditing accounting manage...,desk dradamu ismalerauditing accounting manage...,desk dradamu ismalerauditing accounting manage...,1,1,390
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,"Dear Friend, My name is LOI C.ESTRADA,The wife...",dear friend name loi cestradathe wife mr josep...,dear friend name loi cestradathe wife mr josep...,dear friend name loi cestradathe wife mr josep...,1,1,1507


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Bag of Words vectorizer
# Creating Bag of Words features
bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_bow = bow_vectorizer.transform(data_val['preprocessed_text'])

print(f"Bag of Words shape - Train: {X_train_bow.shape}, Validation: {X_val_bow.shape}")

Bag of Words shape - Train: (800, 1000), Validation: (200, 1000)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Load the vectorizer
print("Creating TF-IDF features...")
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_tfidf = tfidf_vectorizer.transform(data_val['preprocessed_text'])

# print the shape of the vectorized dataset
print(f"TF-IDF shape - Train: {X_train_tfidf.shape}, Validation: {X_val_tfidf.shape}")


Creating TF-IDF features...
TF-IDF shape - Train: (800, 1000), Validation: (200, 1000)


## And the Train a Classifier?

In [ ]:
def train_and_evaluate(X_train, X_val, y_train, y_val, feature_name):
    """
    Train MultinomialNB classifier and evaluate performance
    """
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)
    
    # Predictions
    y_pred = classifier.predict(X_val)
    
    # Evaluation
    accuracy = accuracy_score(y_val, y_pred)
    print(f"\n{feature_name} Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_val, y_pred))
    
    return accuracy



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Prepare target variables
y_train = data_train['label']
y_val = data_val['label']

# Test different feature combinations
results = {}

# 1. Bag of Words only
accuracy_bow = train_and_evaluate(X_train_bow, X_val_bow, y_train, y_val, "Bag of Words Only")
results['Bag of Words Only'] = accuracy_bow

# 2. TF-IDF only
accuracy_tfidf = train_and_evaluate(X_train_tfidf, X_val_tfidf, y_train, y_val, "TF-IDF Only")
results['TF-IDF Only'] = accuracy_tfidf

# 3. Bag of Words + extra features
from scipy.sparse import hstack, csr_matrix
X_train_bow_extra = hstack([X_train_bow, csr_matrix(data_train[['money_mark', 'suspicious_words', 'text_len']].values)])
X_val_bow_extra = hstack([X_val_bow, csr_matrix(data_val[['money_mark', 'suspicious_words', 'text_len']].values)])

accuracy_bow_extra = train_and_evaluate(X_train_bow_extra, X_val_bow_extra, y_train, y_val, "Bag of Words + Extra Features")
results['Bag of Words + Extra Features'] = accuracy_bow_extra

# 4. TF-IDF + extra features
X_train_tfidf_extra = hstack([X_train_tfidf, csr_matrix(data_train[['money_mark', 'suspicious_words', 'text_len']].values)])
X_val_tfidf_extra = hstack([X_val_tfidf, csr_matrix(data_val[['money_mark', 'suspicious_words', 'text_len']].values)])

accuracy_tfidf_extra = train_and_evaluate(X_train_tfidf_extra, X_val_tfidf_extra, y_train, y_val, "TF-IDF + Extra Features")
results['TF-IDF + Extra Features'] = accuracy_tfidf_extra

# Display results comparison
for feature, accuracy in results.items():
    print(f"{feature}: {accuracy:.4f}")

# Find the best performing feature combination
best_feature = max(results, key=results.get)
print(f"\nBest performing feature combination: {best_feature} with accuracy {results[best_feature]:.4f}")



Bag of Words Only Results:
Accuracy: 0.9300
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.88      0.93       112
           1       0.87      0.99      0.93        88

    accuracy                           0.93       200
   macro avg       0.93      0.94      0.93       200
weighted avg       0.94      0.93      0.93       200


TF-IDF Only Results:
Accuracy: 0.9550
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       112
           1       0.92      0.98      0.95        88

    accuracy                           0.95       200
   macro avg       0.95      0.96      0.95       200
weighted avg       0.96      0.95      0.96       200


Bag of Words + Extra Features Results:
Accuracy: 0.9300
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.88      0.93       112
           1       0.87    